<a href="https://colab.research.google.com/github/andrew11morozovtwo/tableau/blob/main/morozov_ab_tabloue_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import math as mth
import plotly.express as px

In [ ]:
dataset = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event.time  74197 non-null  object
 1   event.name  74197 non-null  object
 2   user.id     74197 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [ ]:
sourсes = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_soures.csv')
sourсes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4293 entries, 0 to 4292
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userId  4293 non-null   object
 1   source  4293 non-null   object
dtypes: object(2)
memory usage: 67.2+ KB


## Предобработка данных

### Предобработка данных по обоим файлам.

Пропусков значений в обоих датасетах нет.

In [ ]:
# Приведем в соответствие тип данных в столбце "event.time"
dataset['event.time'] = pd.to_datetime(dataset['event.time'])
# dataset.info() # проверка

In [ ]:
# Изменим названия столбцов в датасете "dataset"
dataset.columns = ['event_time', 'event_name', 'user_id']

In [ ]:
# Изменим названия столбцов в датасете "sourсes"
sourсes.columns = ['user_id', 'source']

In [ ]:
# проверка на дубликаты
for df in [dataset, sourсes]:
    display(df.duplicated().sum())

0

0

Дубликатов нет

### Предобработка по dataset

In [ ]:
dataset.info()
#, sourсes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_time  74197 non-null  datetime64[ns]
 1   event_name  74197 non-null  object        
 2   user_id     74197 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1.7+ MB


In [ ]:
dataset['event_name'].unique()

array(['advert_open', 'tips_show', 'map', 'contacts_show', 'search_4',
       'search_5', 'tips_click', 'photos_show', 'search_1', 'search_2',
       'search_3', 'favorites_add', 'contacts_call', 'search_6',
       'search_7', 'show_contacts'], dtype=object)

In [ ]:
# Приведение событий “contacts_show» и «show_contacts» к «contacts_show»
def rename(i):
    if i == 'show_contacts':
        new_name = 'contacts_show'
        return new_name
    return i

dataset['event_name'] = dataset['event_name'].apply(rename)

In [ ]:
# Уникальные названия событий после переименования
dataset['event_name'].unique()

array(['advert_open', 'tips_show', 'map', 'contacts_show', 'search_4',
       'search_5', 'tips_click', 'photos_show', 'search_1', 'search_2',
       'search_3', 'favorites_add', 'contacts_call', 'search_6',
       'search_7'], dtype=object)

In [ ]:
# Количество событий в наблюдаемый период
t = dataset['event_name'].value_counts()
t

tips_show        40055
photos_show      10012
advert_open       6164
contacts_show     4529
map               3881
search_1          3506
favorites_add     1417
search_5          1049
tips_click         814
search_4           701
contacts_call      541
search_3           522
search_6           460
search_2           324
search_7           222
Name: event_name, dtype: int64

In [ ]:
sourсes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4293 entries, 0 to 4292
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  4293 non-null   object
 1   source   4293 non-null   object
dtypes: object(2)
memory usage: 67.2+ KB


In [ ]:
# Количество уникальных пользователей в sourсes
sourсes['user_id'].nunique()

4293

In [ ]:
# Количество источников
sourсes['source'].nunique()

3

In [ ]:
# Оригинальные названия источников
sourсes['source'].unique()

array(['other', 'yandex', 'google'], dtype=object)

In [ ]:
# Проверка на отсутствие общих элементов
yandex_set = set(df[df['source']=='yandex']['user_id'])
google_set = set(df[df['source']=='google']['user_id'])
other_set = set(df[df['source']=='other']['user_id'])

print(
    'yandex_set не имеет общих элементов c google_set: {}\n'
    'yandex_set не имеет общих элементов c other_set: {}\n'
    'google_set не имеет общих элементов c other_set: {}'.format(yandex_set.isdisjoint(google_set), yandex_set.isdisjoint(other_set), google_set.isdisjoint(other_set)))


yandex_set не имеет общих элементов c google_set: True
yandex_set не имеет общих элементов c other_set: True
google_set не имеет общих элементов c other_set: True


In [ ]:
#Распределение пользователей по источникам
t = sourсes.pivot_table(index= sourсes['source'], aggfunc={'user_id':'nunique'}).sort_values('user_id', ascending=False)
t.sort_values('user_id', ascending=False) #.sort_values('')

,user_id
source,
yandex,1934
other,1230
google,1129


In [ ]:
# функция для вывода гистограммы, чтобы однообразно выводить графики.
def my_bar(df, x, y, title, y_title):
    fig = px.bar(df, x=x, y=y)
    fig.update_xaxes(tickangle=-90, tickmode = 'linear')

    fig.update_layout(title=title,
                      yaxis_title=y_title,
                      margin=dict(l=0, r=0))
    fig.show()

In [ ]:
my_bar(t, t.index, t['user_id'],
      'Распределение количества пользователей по источникам', 'Количество пользователей')

### Результаты предобработки данных

Данные, находящиеся в двух датасетах загружены и проверены на соответствие условиям задания.

Выполнены следующие работы по обработки данных:
- Приведены в соответствия правилам Pandas названия колонок в обоих датасетах;
- Проведена проверка на дубликаты. Дубликаты отсутствуют;
- В datset объедены события “contacts_show» и «show_contacts». События приведены к общему событию «contacts_show».
- Рассмотрены распределения данных внутри датасетов. Аномалий не выявлено.

**Данные готовы к дальнейшим исследования**

In [ ]:
dataset.sample(1)

,event_time,event_name,user_id
16005,2019-10-14 07:09:17.525226,advert_open,21b9ef95-e152-47e6-bb4b-284525c38064


In [ ]:
sourсes.sample(1)

,user_id,source
11,a63a3a79-08fa-4ec2-895c-50b4eb09b8d0,google


Принимаем решение о "склеивании" в одно событие `'search'` событий `search_1 — search_7`, связанных с поиском по сайту

In [ ]:
"""def rename(i):
   if 'search' in i:
    new_name = 'search'
    return new_name
   return i

df['event_name'] = df['event_name'].apply(rename)"""

"def rename(i):\n   if 'search' in i:\n    new_name = 'search'\n    return new_name\n   return i\n\ndf['event_name'] = df['event_name'].apply(rename)"

#### Объединение двух сетов

Для дальнейшей работы по исследованию данных необходимо объеденить данные, находящиеся в двух сетах:

In [ ]:
df = dataset.merge(sourсes, on='user_id', how = 'right')
df

,event_time,event_name,user_id,source
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894,other
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
2,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
3,2019-10-07 00:01:27.770232,advert_open,020292ab-89bc-4156-9acf-68bc2783f894,other
4,2019-10-07 00:01:34.804591,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
...,...,...,...,...
74192,2019-11-03 23:46:47.068179,map,d157bffc-264d-4464-8220-1cc0c42f43a9,google
74193,2019-11-03 23:46:58.914787,advert_open,d157bffc-264d-4464-8220-1cc0c42f43a9,google
74194,2019-11-03 23:47:01.232230,tips_show,d157bffc-264d-4464-8220-1cc0c42f43a9,google
74195,2019-11-03 23:47:47.475102,advert_open,d157bffc-264d-4464-8220-1cc0c42f43a9,google


#### Проверки после объединения

In [ ]:
# Проверка на дубликаты
df.duplicated().sum()

0

In [ ]:
# проверка на пропуски, соответствие типов
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74197 entries, 0 to 74196
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_time  74197 non-null  datetime64[ns]
 1   event_name  74197 non-null  object        
 2   user_id     74197 non-null  object        
 3   source      74197 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.8+ MB


После объединения таблиц важно убедиться, что не возникло пользователей с двойным источником.

In [ ]:
# Проверка на отсутствие общих элементов
yandex_set = set(df[df['source']=='yandex']['user_id'])
google_set = set(df[df['source']=='google']['user_id'])
other_set = set(df[df['source']=='other']['user_id'])

print(
    'yandex_set не имеет общих элементов c google_set: {}\n'
    'yandex_set не имеет общих элементов c other_set: {}\n'
    'google_set не имеет общих элементов c other_set: {}'.format(yandex_set.isdisjoint(google_set), yandex_set.isdisjoint(other_set), google_set.isdisjoint(other_set)))


yandex_set не имеет общих элементов c google_set: True
yandex_set не имеет общих элементов c other_set: True
google_set не имеет общих элементов c other_set: True


In [ ]:
df

,event_time,event_name,user_id,source
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894,other
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
2,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
3,2019-10-07 00:01:27.770232,advert_open,020292ab-89bc-4156-9acf-68bc2783f894,other
4,2019-10-07 00:01:34.804591,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
...,...,...,...,...
74192,2019-11-03 23:46:47.068179,map,d157bffc-264d-4464-8220-1cc0c42f43a9,google
74193,2019-11-03 23:46:58.914787,advert_open,d157bffc-264d-4464-8220-1cc0c42f43a9,google
74194,2019-11-03 23:47:01.232230,tips_show,d157bffc-264d-4464-8220-1cc0c42f43a9,google
74195,2019-11-03 23:47:47.475102,advert_open,d157bffc-264d-4464-8220-1cc0c42f43a9,google


## Выполнение проекта для Табло

In [ ]:
# Выгрузка файла на Google Disc
from google.colab import  drive
drive.mount('/drive')
df.to_csv('/drive/My Drive/dash_df.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
#dash_visits.to_csv('/drive/My Drive/dash_visits1.csv')